In [155]:
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor, Wav2Vec2ForCTC
from IPython.display import Audio
from tqdm import tqdm
import moviepy as mp
import torch
import librosa
import os
import yt_dlp
import textwrap

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [156]:
model = Wav2Vec2ForCTC.from_pretrained(
    'facebook/wav2vec2-large-960h', 
    torch_dtype=torch.float16
    ).to(device)

processor = Wav2Vec2Processor.from_pretrained('facebook/wav2vec2-large-960h')  


Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [157]:
clip = 'https://www.youtube.com/shorts/ePepT1HdOpo'

start = 0
end = 10000

In [158]:
download_opts = {
    'format': 'worst',  # Download the worst quality video
    'recode_video': 'mp4',  # Recode to mp4 format
    'outtmpl': 'clip.%(ext)s',  # Save as 'clip.mp4'
    'overwrites': True,
}

with yt_dlp.YoutubeDL(download_opts) as ydl:
    ydl.download([clip])

[youtube] Extracting URL: https://www.youtube.com/shorts/ePepT1HdOpo
[youtube] ePepT1HdOpo: Downloading webpage
[youtube] ePepT1HdOpo: Downloading ios player API JSON
[youtube] ePepT1HdOpo: Downloading mweb player API JSON
[youtube] ePepT1HdOpo: Downloading m3u8 information
[info] ePepT1HdOpo: Downloading 1 format(s): 18
Deleting existing file clip.mp4
[download] Destination: clip.mp4
[download] 100% of    4.23MiB in 00:00:00 at 19.30MiB/s  


In [159]:
clip = mp.VideoFileClip("clip.mp4")
end = min(clip.duration, end)

# Save the paths for later
clip_paths = []

# Extract Audio-only from mp4
for i in range(start, int(end), 10):
  sub_end = min(i+10, end)
  sub_clip = clip[i: sub_end]

  sub_clip.audio.write_audiofile("audio_" + str(i) + ".mp3")
  clip_paths.append("audio_" + str(i) + ".mp3")

MoviePy - Writing audio in audio_0.mp3


MoviePy - Done.


MoviePy - Writing audio in audio_10.mp3


MoviePy - Done.


MoviePy - Writing audio in audio_20.mp3


...

MoviePy - Done.


MoviePy - Writing audio in audio_40.mp3


MoviePy - Done.


MoviePy - Writing audio in audio_50.mp3


MoviePy - Done.


In [160]:
# Play Audio 
Audio(clip_paths[len(clip_paths) - 1])

In [161]:
cc = ''

for path in tqdm(clip_paths):
    input_audio, _ = librosa.load(path, sr=16000)
    # print(input_audio)
    
    
    input_values = processor(input_audio, return_tensors="pt", padding="longest").input_values.to(torch.float16)
    
    with torch.no_grad():
        logits = model(input_values).logits
        predicted_ids = torch.argmax(logits, dim=-1)
        
    transcription = processor.batch_decode(predicted_ids)[0]
    cc += transcription 
    cc += ' '


100%|██████████| 6/6 [01:38<00:00, 16.46s/it]


In [162]:
wrapped_text = textwrap.fill(cc, width=150)
print(wrapped_text)

WE'VE BUILT THIS BEAUTIFUL GLORIOUS SPACE PLATFORM FASTER THAN ANYTHING I'VE EVER MADE MORE CAPACITY THAN ANYTHING ELSE HAVE EVER MADE MORE FIRE POWER
THEN AIN'T WELL OIL CHALIS THERE'S SOMETHINGS HAVE MORE FIREPOWER ZEUS HAS MORE FIRE POWER BUT IF I NEED TO DO ER A NEW VERSION OF ZEUS I COULD DO
BETTER THAN THIS NOW THAT I KNOW WHAT THE REQUIREMENTS ARE AND THEN THINGS HAVE CHANGED A LITTLE BIT THE GAMEPLATE O WE DIDN'T GET A FULL LOAD OF
METHIUM TAM YET NOW THE NIGHT I KNOW MORE ABOUT WHAT IT REQUIRES TO DO ER TO DO THE YUN PROMETHIUM RUNS I'VE GOT SOME IDEAS IF I HAVE TO REDESIGN THE
PLATFORM ARABESA NEEDS TO START FROM THE GROUND BUT IT'S GOING TO BE IT'S SUMMER THAT I CAN USE SOME OF THE PRINCIPLES FROM ARTIMIS THIS ME THOD OF
PROCESSING ASTEROID TO THINK IS REALLY GOOD I LIKE THIS AND NOW WE KNOW NOW WE'VE GOT THE HOLE LIKE PROGRAMMING PILTERS ON ER ON GRABBERS THAT'S
FANTASTIC
